In [2]:
import numpy as np
import h5py
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#drive.mount('/content/drive')

# Function to extract features from video frames
def extract_features_from_video(video_path, target_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame
        frame = cv2.resize(frame, target_size)
        # Normalize pixel values
        frame = frame / 255.0
        features.append(frame)
    cap.release()
    return features

# Function to load dataset
def load_dataset(data_dir, max_frames=100):
    X = []
    y = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
                if video_file.endswith('.mp4'):  # Assuming videos are in .mp4 format
                    video_path = os.path.join(class_dir, video_file)
                    features = extract_features_from_video(video_path)
                    # Pad or truncate the sequences to max_frames
                    if len(features) < max_frames:
                        features.extend([np.zeros_like(features[0])] * (max_frames - len(features)))
                    elif len(features) > max_frames:
                        features = features[:max_frames]
                    X.append(features)
                    y.append(class_name)
    return np.array(X), np.array(y)

# Navigate to the directory containing the dataset
os.chdir("/content/drive/MyDrive/Capstone/URFD")

# Load dataset
X, y = load_dataset("/content/drive/MyDrive/Capstone/URFD")

In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X,y

(array([[[[[0.34901961, 0.34901961, 0.34901961],
           [0.44705882, 0.44705882, 0.44705882],
           [0.43529412, 0.43529412, 0.43529412],
           ...,
           [0.53333333, 0.60392157, 0.70196078],
           [0.20784314, 0.20784314, 0.2745098 ],
           [0.24705882, 0.2627451 , 0.30588235]],
 
          [[0.35686275, 0.35686275, 0.35686275],
           [0.43137255, 0.43137255, 0.43137255],
           [0.43529412, 0.43529412, 0.43529412],
           ...,
           [0.50196078, 0.59215686, 0.70588235],
           [0.21176471, 0.28627451, 0.35686275],
           [0.39215686, 0.43137255, 0.49411765]],
 
          [[0.09019608, 0.09019608, 0.09019608],
           [0.42745098, 0.42745098, 0.42745098],
           [0.43529412, 0.43529412, 0.43529412],
           ...,
           [0.15294118, 0.21176471, 0.32941176],
           [0.27058824, 0.36078431, 0.38431373],
           [0.34901961, 0.40392157, 0.42352941]],
 
          ...,
 
          [[0.27843137, 0.27843137, 0.278431

In [7]:
model = Sequential()
model.add(layers.Conv3D(32, (3, 3, 3), activation='relu',input_shape=(100, 64, 64, 3)))
model.add(layers.MaxPooling3D((2, 2, 2)))
model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
model.add(layers.MaxPooling3D((2, 2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 98, 62, 62, 32)    2624      
                                                                 
 max_pooling3d (MaxPooling3  (None, 49, 31, 31, 32)    0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 47, 29, 29, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 23, 14, 14, 64)    0         
 g3D)                                                            
                                                                 
 flatten (Flatten)           (None, 288512)            0         
                                                                 
 dense (Dense)               (None, 1)                 2

In [8]:
model.compile(Adam(0.001), 'binary_crossentropy', ['accuracy'])
model.fit(X_train,y_train,epochs=10,batch_size=32,verbose=1,validation_data=(X_test,y_test),callbacks=[EarlyStopping(patience=15)])
#Testing the 3D-CNN
_, acc = model.evaluate(X_train, y_train)
print('training accuracy:', str(round(acc*100, 2))+'%')
_, acc = model.evaluate(X_test, y_test)
print('testing accuracy:', str(round(acc*100, 2))+'%')

Epoch 1/10
2/2 [==============================] - 113s 50s/step - loss: 0.7826 - accuracy: 0.4643 - val_loss: 1.6708 - val_accuracy: 0.2857
Epoch 2/10
2/2 [==============================] - 107s 50s/step - loss: 2.0827 - accuracy: 0.4643 - val_loss: 0.5201 - val_accuracy: 0.2857
Epoch 3/10
2/2 [==============================] - 106s 49s/step - loss: 0.3441 - accuracy: 0.4643 - val_loss: 0.1505 - val_accuracy: 0.2857
Epoch 4/10
2/2 [==============================] - 111s 54s/step - loss: 0.1838 - accuracy: 0.4643 - val_loss: 0.0707 - val_accuracy: 0.2857
Epoch 5/10
2/2 [==============================] - 107s 51s/step - loss: 0.0490 - accuracy: 0.4643 - val_loss: 0.0364 - val_accuracy: 0.2857
Epoch 6/10
2/2 [==============================] - 110s 53s/step - loss: 0.0265 - accuracy: 0.4643 - val_loss: 0.0027 - val_accuracy: 0.2857
Epoch 7/10
2/2 [==============================] - 109s 53s/step - loss: 0.0038 - accuracy: 0.4643 - val_loss: 0.0058 - val_accuracy: 0.2857
Epoch 8/10
2/2 [====